1 subject. Feature extraction. Classification for each frequency band.

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
import csv

#def select_file(file):
    #Lectura mediante pandas
f = "data_preprocessed_python\\s01.dat"
subject_id = 1

dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)
#df_videos

In [40]:
    # Frecuencia de muestreo
    fs = 128
    # Window
    window = "hann"
    # Length of each segment
    # nperseg = 256 por defecto
    # noverlap
    # Por defecto a None, if None : noverlap = nperseg / 2
    # [...]

    # Definicion de bandas
    eeg_bands = {'Delta': (1, 4),
                'Theta': (4, 8),
                'Alpha': (8, 14),
                'Beta': (14, 30),
                'Gamma': (30, 50)}

In [41]:
# FOURIER
import scipy
from scipy import signal

# SE COGE EL PRIMER VÍDEOS, Df_VIDEOS[0]
def do_stft(video, channel):
    # Array 
    x = df_videos[video][channel]

    # Array of the sample frequency, Array of the segment times, STFT of x
    f, t , Zxx = scipy.signal.stft(x, fs, window)

    #print(np.abs(Zxx))
    '''
    plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    '''
    return Zxx


In [42]:
#SEPARACION EN LAS DISTINTAS BANDAS DE FRECUENCIA

def make_bands(Zxx):
    #Obtener valores reales de STFT, solo positivos.
    values = np.absolute(Zxx)

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
    #print(fft_freq)

    # Obtención de cada una de las bandas de frecuencia
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = values[freq_ix]

    #print(eeg_band_fft)
    '''
    eeg_band_fft_means = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft_means[band] = np.mean(values[freq_ix])

    #print(eeg_band_fft_means)

    #Graficación de las medias de los datos
    df = pd.DataFrame(columns=['band', 'val'])
    df['band'] = eeg_bands.keys()
    df['val'] = [eeg_band_fft_means[band] for band in eeg_bands]
    ax = df.plot.bar(x='band', y='val', legend=False)
    ax.set_xlabel("EEG band")
    ax.set_ylabel("Mean band Amplitude")
    '''
    #print(df)

    #print("\n\n")

    '''
    print("Shape de Zxx: ", Zxx.shape)
    print("Shape de Delta: ", eeg_band_fft['Delta'].shape)
    print("Shape de Theta: ", eeg_band_fft['Theta'].shape)
    print("Shape de Alpha: ", eeg_band_fft['Alpha'].shape)
    print("Shape de Beta: ", eeg_band_fft['Beta'].shape)
    print("Shape de Gamma: ", eeg_band_fft['Gamma'].shape)
    '''
    return eeg_band_fft


In [43]:
# Ahora hay que hacer las medias sobre cada banda de frecuencia. La media entre cada una de las posiciones de los bloques. Hacer la media entre todas las posiciones 0, 1, 2... 63 de cada uno de los bloques
import numpy as np

def channel_freq(eeg_band_fft):
        
        freq = np.zeros((5, 64))        # 5 bandas, 64 posiciones de los datos

        # Se va a recorrer {Delta, Theta, Alpha, Beta, Gamma}
        for band in eeg_bands:

                # Para cada una de las 64 posiciones del array
                for j in range (0, 64):
                        
                        val = []
                        # Se va a recorrer cada uno de los arrays que hay en cada una de las bandas
                        for i in range (0, len(eeg_band_fft[band])):
                                val.append(eeg_band_fft[band][i][j])
                                # Ejemplo : eeg_band_fft['Delta'][0][0*0 + 0]
                        if (band == "Delta"):
                                freq[0][j] = np.mean(val)
                        elif (band == "Theta"):
                                freq[1][j] = np.mean(val)
                        elif (band == "Alpha"):
                                freq[2][j] = np.mean(val)
                        elif (band == "Beta"):
                                freq[3][j] = np.mean(val)
                        elif (band == "Gamma"):
                                freq[4][j] = np.mean(val)
        return(freq)     # En freq tenemos las frecuencias medias de cada una de las bandas

# Recordamos : Estamos tratando los datos relativos al video 0, un canal concreto de un sujeto concreto

In [44]:
#df_file_videos = select_file('d\\s01.dat')
# Obtener la división por bandas de frecuencia de cada uno de los canales del video 0
for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

In [45]:
# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

aux_delta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.023039,0.024048,0.028811,0.032106,0.044012,0.048092,0.016513,0.050341,0.035829,0.011347,...,0.072781,0.022990,0.019260,0.010448,0.021503,0.038031,0.035371,0.003149,0.029362,0.033688
1,0.023039,0.024048,0.028811,0.032106,0.044012,0.048092,0.016513,0.050341,0.035829,0.011347,...,0.072781,0.022990,0.019260,0.010448,0.021503,0.038031,0.035371,0.003149,0.029362,0.033688
2,0.023039,0.024048,0.028811,0.032106,0.044012,0.048092,0.016513,0.050341,0.035829,0.011347,...,0.072781,0.022990,0.019260,0.010448,0.021503,0.038031,0.035371,0.003149,0.029362,0.033688
3,0.023039,0.024048,0.028811,0.032106,0.044012,0.048092,0.016513,0.050341,0.035829,0.011347,...,0.072781,0.022990,0.019260,0.010448,0.021503,0.038031,0.035371,0.003149,0.029362,0.033688
4,0.023039,0.024048,0.028811,0.032106,0.044012,0.048092,0.016513,0.050341,0.035829,0.011347,...,0.072781,0.022990,0.019260,0.010448,0.021503,0.038031,0.035371,0.003149,0.029362,0.033688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.025769,0.026977,0.056836,0.010858,0.015764,0.051700,0.006908,0.011550,0.038232,0.034416,...,0.012704,0.008985,0.026565,0.012750,0.029309,0.025757,0.056188,0.026164,0.045488,0.014212
2556,0.025769,0.026977,0.056836,0.010858,0.015764,0.051700,0.006908,0.011550,0.038232,0.034416,...,0.012704,0.008985,0.026565,0.012750,0.029309,0.025757,0.056188,0.026164,0.045488,0.014212
2557,0.025769,0.026977,0.056836,0.010858,0.015764,0.051700,0.006908,0.011550,0.038232,0.034416,...,0.012704,0.008985,0.026565,0.012750,0.029309,0.025757,0.056188,0.026164,0.045488,0.014212
2558,0.025769,0.026977,0.056836,0.010858,0.015764,0.051700,0.006908,0.011550,0.038232,0.034416,...,0.012704,0.008985,0.026565,0.012750,0.029309,0.025757,0.056188,0.026164,0.045488,0.014212


In [46]:
# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

aux_theta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.017252,0.018866,0.023816,0.027844,0.050185,0.061815,0.016491,0.057747,0.031317,0.007595,...,0.075730,0.02869,0.020661,0.013362,0.023908,0.042528,0.036502,0.002304,0.029287,0.032271
1,0.017252,0.018866,0.023816,0.027844,0.050185,0.061815,0.016491,0.057747,0.031317,0.007595,...,0.075730,0.02869,0.020661,0.013362,0.023908,0.042528,0.036502,0.002304,0.029287,0.032271
2,0.017252,0.018866,0.023816,0.027844,0.050185,0.061815,0.016491,0.057747,0.031317,0.007595,...,0.075730,0.02869,0.020661,0.013362,0.023908,0.042528,0.036502,0.002304,0.029287,0.032271
3,0.017252,0.018866,0.023816,0.027844,0.050185,0.061815,0.016491,0.057747,0.031317,0.007595,...,0.075730,0.02869,0.020661,0.013362,0.023908,0.042528,0.036502,0.002304,0.029287,0.032271
4,0.017252,0.018866,0.023816,0.027844,0.050185,0.061815,0.016491,0.057747,0.031317,0.007595,...,0.075730,0.02869,0.020661,0.013362,0.023908,0.042528,0.036502,0.002304,0.029287,0.032271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.033588,0.035551,0.063285,0.008209,0.019997,0.064907,0.005452,0.008309,0.041939,0.032092,...,0.020028,0.01476,0.027922,0.015252,0.031851,0.031492,0.059141,0.030113,0.048370,0.018528
2556,0.033588,0.035551,0.063285,0.008209,0.019997,0.064907,0.005452,0.008309,0.041939,0.032092,...,0.020028,0.01476,0.027922,0.015252,0.031851,0.031492,0.059141,0.030113,0.048370,0.018528
2557,0.033588,0.035551,0.063285,0.008209,0.019997,0.064907,0.005452,0.008309,0.041939,0.032092,...,0.020028,0.01476,0.027922,0.015252,0.031851,0.031492,0.059141,0.030113,0.048370,0.018528
2558,0.033588,0.035551,0.063285,0.008209,0.019997,0.064907,0.005452,0.008309,0.041939,0.032092,...,0.020028,0.01476,0.027922,0.015252,0.031851,0.031492,0.059141,0.030113,0.048370,0.018528


In [47]:
#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

aux_alpha

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.019937,0.022854,0.029019,0.034244,0.054900,0.069261,0.018295,0.062502,0.032166,0.008225,...,0.082650,0.034306,0.027026,0.019712,0.026368,0.046145,0.041746,0.008357,0.032070,0.039993
1,0.019937,0.022854,0.029019,0.034244,0.054900,0.069261,0.018295,0.062502,0.032166,0.008225,...,0.082650,0.034306,0.027026,0.019712,0.026368,0.046145,0.041746,0.008357,0.032070,0.039993
2,0.019937,0.022854,0.029019,0.034244,0.054900,0.069261,0.018295,0.062502,0.032166,0.008225,...,0.082650,0.034306,0.027026,0.019712,0.026368,0.046145,0.041746,0.008357,0.032070,0.039993
3,0.019937,0.022854,0.029019,0.034244,0.054900,0.069261,0.018295,0.062502,0.032166,0.008225,...,0.082650,0.034306,0.027026,0.019712,0.026368,0.046145,0.041746,0.008357,0.032070,0.039993
4,0.019937,0.022854,0.029019,0.034244,0.054900,0.069261,0.018295,0.062502,0.032166,0.008225,...,0.082650,0.034306,0.027026,0.019712,0.026368,0.046145,0.041746,0.008357,0.032070,0.039993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.033685,0.037036,0.067088,0.007560,0.024652,0.071946,0.006097,0.009998,0.041666,0.039836,...,0.035355,0.023049,0.034356,0.023025,0.036246,0.036683,0.066209,0.039338,0.054329,0.023807
2556,0.033685,0.037036,0.067088,0.007560,0.024652,0.071946,0.006097,0.009998,0.041666,0.039836,...,0.035355,0.023049,0.034356,0.023025,0.036246,0.036683,0.066209,0.039338,0.054329,0.023807
2557,0.033685,0.037036,0.067088,0.007560,0.024652,0.071946,0.006097,0.009998,0.041666,0.039836,...,0.035355,0.023049,0.034356,0.023025,0.036246,0.036683,0.066209,0.039338,0.054329,0.023807
2558,0.033685,0.037036,0.067088,0.007560,0.024652,0.071946,0.006097,0.009998,0.041666,0.039836,...,0.035355,0.023049,0.034356,0.023025,0.036246,0.036683,0.066209,0.039338,0.054329,0.023807


In [48]:
# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

aux_beta

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.074772,0.122948,0.120034,0.094305,0.123077,0.200693,0.060057,0.098501,0.065002,0.030612,...,0.162639,0.106128,0.110137,0.068395,0.060210,0.102504,0.091276,0.113631,0.096028,0.073698
1,0.074772,0.122948,0.120034,0.094305,0.123077,0.200693,0.060057,0.098501,0.065002,0.030612,...,0.162639,0.106128,0.110137,0.068395,0.060210,0.102504,0.091276,0.113631,0.096028,0.073698
2,0.074772,0.122948,0.120034,0.094305,0.123077,0.200693,0.060057,0.098501,0.065002,0.030612,...,0.162639,0.106128,0.110137,0.068395,0.060210,0.102504,0.091276,0.113631,0.096028,0.073698
3,0.074772,0.122948,0.120034,0.094305,0.123077,0.200693,0.060057,0.098501,0.065002,0.030612,...,0.162639,0.106128,0.110137,0.068395,0.060210,0.102504,0.091276,0.113631,0.096028,0.073698
4,0.074772,0.122948,0.120034,0.094305,0.123077,0.200693,0.060057,0.098501,0.065002,0.030612,...,0.162639,0.106128,0.110137,0.068395,0.060210,0.102504,0.091276,0.113631,0.096028,0.073698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.040818,0.105418,0.185211,0.107109,0.100188,0.242917,0.101689,0.052823,0.069783,0.167223,...,0.183980,0.171910,0.176778,0.226287,0.159771,0.090470,0.156451,0.214108,0.127266,0.111628
2556,0.040818,0.105418,0.185211,0.107109,0.100188,0.242917,0.101689,0.052823,0.069783,0.167223,...,0.183980,0.171910,0.176778,0.226287,0.159771,0.090470,0.156451,0.214108,0.127266,0.111628
2557,0.040818,0.105418,0.185211,0.107109,0.100188,0.242917,0.101689,0.052823,0.069783,0.167223,...,0.183980,0.171910,0.176778,0.226287,0.159771,0.090470,0.156451,0.214108,0.127266,0.111628
2558,0.040818,0.105418,0.185211,0.107109,0.100188,0.242917,0.101689,0.052823,0.069783,0.167223,...,0.183980,0.171910,0.176778,0.226287,0.159771,0.090470,0.156451,0.214108,0.127266,0.111628


In [49]:
# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

aux_gamma

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.234008,0.333754,0.327015,0.347129,0.342250,0.403499,0.239339,0.170319,0.185646,0.147057,...,0.596176,0.299382,0.389189,0.198125,0.254384,0.197694,0.217398,0.300440,0.373761,0.425536
1,0.234008,0.333754,0.327015,0.347129,0.342250,0.403499,0.239339,0.170319,0.185646,0.147057,...,0.596176,0.299382,0.389189,0.198125,0.254384,0.197694,0.217398,0.300440,0.373761,0.425536
2,0.234008,0.333754,0.327015,0.347129,0.342250,0.403499,0.239339,0.170319,0.185646,0.147057,...,0.596176,0.299382,0.389189,0.198125,0.254384,0.197694,0.217398,0.300440,0.373761,0.425536
3,0.234008,0.333754,0.327015,0.347129,0.342250,0.403499,0.239339,0.170319,0.185646,0.147057,...,0.596176,0.299382,0.389189,0.198125,0.254384,0.197694,0.217398,0.300440,0.373761,0.425536
4,0.234008,0.333754,0.327015,0.347129,0.342250,0.403499,0.239339,0.170319,0.185646,0.147057,...,0.596176,0.299382,0.389189,0.198125,0.254384,0.197694,0.217398,0.300440,0.373761,0.425536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0.059516,0.157184,0.216002,0.247524,0.108213,0.300691,0.210246,0.229857,0.294505,0.185836,...,0.254134,0.208087,0.180113,0.467545,0.298015,0.161810,0.274674,0.369125,0.256507,0.241600
2556,0.059516,0.157184,0.216002,0.247524,0.108213,0.300691,0.210246,0.229857,0.294505,0.185836,...,0.254134,0.208087,0.180113,0.467545,0.298015,0.161810,0.274674,0.369125,0.256507,0.241600
2557,0.059516,0.157184,0.216002,0.247524,0.108213,0.300691,0.210246,0.229857,0.294505,0.185836,...,0.254134,0.208087,0.180113,0.467545,0.298015,0.161810,0.274674,0.369125,0.256507,0.241600
2558,0.059516,0.157184,0.216002,0.247524,0.108213,0.300691,0.210246,0.229857,0.294505,0.185836,...,0.254134,0.208087,0.180113,0.467545,0.298015,0.161810,0.274674,0.369125,0.256507,0.241600


In [50]:
df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

aux_labels

,valence,arousal,dominance,liking
0,7.71,7.60,6.90,7.83
1,7.71,7.60,6.90,7.83
2,7.71,7.60,6.90,7.83
3,7.71,7.60,6.90,7.83
4,7.71,7.60,6.90,7.83
...,...,...,...,...
2555,5.10,7.12,6.17,5.97
2556,5.10,7.12,6.17,5.97
2557,5.10,7.12,6.17,5.97
2558,5.10,7.12,6.17,5.97


In [51]:
data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

In [52]:
data = data_delta.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']
'''
label = np.where((data['valence'] <4.5) & (data['arousal']<4.5), 'sadness')
label = np.where((data['valence'] <4.5) & (data['arousal'] > 4.5), 'pleasure')
label = np.where((data['valence'] >4.5) & (data['arousal']<4.5), 'anger')
label = np.where((data['valence'] >4.5) & (data['arousal'] > 4.5), 'joy')
'''
data['valence'] = np.select(conditions, choices, default='black')
data = data.drop(columns='arousal')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz', 'label']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia['label']

x = copia.drop(columns=['label'])

    # test_size = 0.2 --> 80% datos para entrenamiento, 20% para test
    # max_depth --> The deeper the tree, the more splits it has and it captures more information about the data. En este modelo, valores altos de max_depths tienden a 'overfit' los datos, lo comprobamos en el siguiente bloque
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

param_distribs_rf = {
    'max_depth': [7, 10, 12]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [5.0, 10.0, 25.0]
}

param_distribs_knn = {
    'n_neighbors': [5,7,9,12,13,15]
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment5_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

from sklearn.metrics import precision_score

rf_prec_sadness_delta = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure_delta = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger_delta = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy_delta = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness_delta = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure_delta = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger_delta = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy_delta = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness_delta = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure_delta = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger_delta = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy_delta = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness_delta = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure_delta = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger_delta = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy_delta = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness_delta = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure_delta = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger_delta = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy_delta = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness_delta = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure_delta = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger_delta = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy_delta = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness_delta = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure_delta = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger_delta = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy_delta = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness_delta = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure_delta = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger_delta = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy_delta = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness_delta = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure_delta = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger_delta = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy_delta = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

NameError: name 'GridSearchCV' is not defined

In [ ]:
data = data_theta.drop(['dominance', 'liking'], axis=1)
data['valence'] = np.select(conditions, choices, default='black')
data = data.drop(columns='arousal')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz', 'label']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia['label']

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

param_distribs_rf = {
    'max_depth': [7, 10, 12]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [5.0, 10.0, 25.0]
}

param_distribs_knn = {
    'n_neighbors': [5,7,9,12,13,15]
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment5_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_theta = accuracy_score(yTest, predict)
svm_acc_theta = accuracy_score(yTest, svm_predict)
knn_acc_theta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness_theta = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure_theta = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger_theta = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy_theta = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness_theta = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure_theta = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger_theta = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy_theta = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness_theta = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure_theta = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger_theta = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy_theta = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness_theta = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure_theta = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger_theta = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy_theta = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness_theta = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure_theta = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger_theta = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy_theta = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness_theta = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure_theta = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger_theta = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy_theta = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness_theta = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure_theta = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger_theta = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy_theta = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness_theta = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure_theta = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger_theta = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy_theta = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness_theta = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure_theta = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger_theta = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy_theta = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python39\lib\

In [ ]:
data = data_alpha.drop(['dominance', 'liking'], axis=1)
data['valence'] = np.select(conditions, choices, default='black')
data = data.drop(columns='arousal')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz', 'label']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia['label']

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

param_distribs_rf = {
    'max_depth': [7, 10, 12]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [5.0, 10.0, 25.0]
}

param_distribs_knn = {
    'n_neighbors': [5,7,9,12,13,15]
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment5_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_alpha = accuracy_score(yTest, predict)
svm_acc_alpha = accuracy_score(yTest, svm_predict)
knn_acc_alpha = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness_alpha = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure_alpha = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger_alpha = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy_alpha = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness_alpha = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure_alpha = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger_alpha = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy_alpha = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness_alpha = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure_alpha = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger_alpha = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy_alpha = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness_alpha = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure_alpha = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger_alpha = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy_alpha = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness_alpha = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure_alpha = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger_alpha = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy_alpha = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness_alpha = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure_alpha = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger_alpha = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy_alpha = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness_alpha = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure_alpha = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger_alpha = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy_alpha = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness_alpha = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure_alpha = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger_alpha = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy_alpha = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness_alpha = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure_alpha = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger_alpha = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy_alpha = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python39\lib\

In [ ]:
data = data_beta.drop(['dominance', 'liking'], axis=1)
data['valence'] = np.select(conditions, choices, default='black')
data = data.drop(columns='arousal')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz', 'label']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia['label']

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

cparam_distribs_rf = {
    'max_depth': [7, 10, 12]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [5.0, 10.0, 25.0]
}

param_distribs_knn = {
    'n_neighbors': [5,7,9,12,13,15]
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment5_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_beta = accuracy_score(yTest, predict)
svm_acc_beta = accuracy_score(yTest, svm_predict)
knn_acc_beta = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness_beta = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure_beta = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger_beta = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy_beta = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness_beta = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure_beta = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger_beta = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy_beta = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness_beta = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure_beta = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger_beta = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy_beta = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness_beta = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure_beta = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger_beta = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy_beta = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness_beta = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure_beta = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger_beta = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy_beta = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness_beta = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure_beta = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger_beta = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy_beta = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness_beta = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure_beta = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger_beta = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy_beta = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness_beta = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure_beta = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger_beta = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy_beta = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness_beta = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure_beta = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger_beta = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy_beta = f1_score(yTest, knn_predict, average="micro", labels=["joy"])     

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
data = data_gamma.drop(['dominance', 'liking'], axis=1)
data['valence'] = np.select(conditions, choices, default='black')
data = data.drop(columns='arousal')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz', 'label']

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia['label']

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

param_distribs_rf = {
    'max_depth': [7, 10, 12]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [5.0, 10.0, 25.0]
}

param_distribs_knn = {
    'n_neighbors': [5,7,9,12,13,15]
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment5_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc_gamma = accuracy_score(yTest, predict)
svm_acc_gamma = accuracy_score(yTest, svm_predict)
knn_acc_gamma = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness_gamma = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure_gamma = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger_gamma = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy_gamma = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness_gamma = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure_gamma = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger_gamma = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy_gamma = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness_gamma = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure_gamma = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger_gamma = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy_gamma = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness_gamma = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure_gamma = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger_gamma = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy_gamma = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness_gamma = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure_gamma = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger_gamma = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy_gamma = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness_gamma = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure_gamma = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger_gamma = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy_gamma = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness_gamma = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure_gamma = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger_gamma = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy_gamma = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness_gamma = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure_gamma = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger_gamma = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy_gamma = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness_gamma = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure_gamma = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger_gamma = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy_gamma = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Sadness precision',
             'Pleasure precision',
             'Anger precision',
             'Joy precision',
             'Sadness recall',
             'Pleasure recall',
             'Anger recall',
             'Joy recall',
             'Sadness f1-score',
             'Pleasure f1-score',
             'Anger f1-score',
             'Joy f1-score']

data_CSV = [subject_id, 5, "STFT", "Delta", "-", "Random Forest", rf_acc_delta, rf_prec_sadness_delta, rf_prec_pleasure_delta, rf_prec_anger_delta, rf_prec_joy_delta, rf_rec_sadness_delta, rf_rec_pleasure_delta, rf_rec_anger_delta, rf_rec_joy_delta, rf_f1_sadness_delta, rf_f1_pleasure_delta, rf_f1_anger_delta, rf_f1_joy_delta], [subject_id, 5, "STFT", "Delta", "-", "SVM", svm_acc_delta, svm_prec_sadness_delta, svm_prec_pleasure_delta, svm_prec_anger_delta, svm_prec_joy_delta, svm_rec_sadness_delta, svm_rec_pleasure_delta, svm_rec_anger_delta, svm_rec_joy_delta, svm_f1_sadness_delta, svm_f1_pleasure_delta, svm_f1_anger_delta, svm_f1_joy_delta], [subject_id, 5, "STFT", "Delta", "-", "kNN", knn_acc_delta, knn_prec_sadness_delta, knn_prec_pleasure_delta, knn_prec_anger_delta, knn_prec_joy_delta, knn_rec_sadness_delta, knn_rec_pleasure_delta, knn_rec_anger_delta, knn_rec_joy_delta, knn_f1_sadness_delta, knn_f1_pleasure_delta, knn_f1_anger_delta, knn_f1_joy_delta], [subject_id, 5, "STFT", "theta", "-", "Random Forest", rf_acc_theta, rf_prec_sadness_theta, rf_prec_pleasure_theta, rf_prec_anger_theta, rf_prec_joy_theta, rf_rec_sadness_theta, rf_rec_pleasure_theta, rf_rec_anger_theta, rf_rec_joy_theta, rf_f1_sadness_theta, rf_f1_pleasure_theta, rf_f1_anger_theta, rf_f1_joy_theta], [subject_id, 5, "STFT", "theta", "-", "SVM", svm_acc_theta, svm_prec_sadness_theta, svm_prec_pleasure_theta, svm_prec_anger_theta, svm_prec_joy_theta, svm_rec_sadness_theta, svm_rec_pleasure_theta, svm_rec_anger_theta, svm_rec_joy_theta, svm_f1_sadness_theta, svm_f1_pleasure_theta, svm_f1_anger_theta, svm_f1_joy_theta], [subject_id, 5, "STFT", "theta", "-", "kNN", knn_acc_theta, knn_prec_sadness_theta, knn_prec_pleasure_theta, knn_prec_anger_theta, knn_prec_joy_theta, knn_rec_sadness_theta, knn_rec_pleasure_theta, knn_rec_anger_theta, knn_rec_joy_theta, knn_f1_sadness_theta, knn_f1_pleasure_theta, knn_f1_anger_theta, knn_f1_joy_theta], [subject_id, 5, "STFT", "alpha", "-", "Random Forest", rf_acc_alpha, rf_prec_sadness_alpha, rf_prec_pleasure_alpha, rf_prec_anger_alpha, rf_prec_joy_alpha, rf_rec_sadness_alpha, rf_rec_pleasure_alpha, rf_rec_anger_alpha, rf_rec_joy_alpha, rf_f1_sadness_alpha, rf_f1_pleasure_alpha, rf_f1_anger_alpha, rf_f1_joy_alpha], [subject_id, 5, "STFT", "alpha", "-", "SVM", svm_acc_alpha, svm_prec_sadness_alpha, svm_prec_pleasure_alpha, svm_prec_anger_alpha, svm_prec_joy_alpha, svm_rec_sadness_alpha, svm_rec_pleasure_alpha, svm_rec_anger_alpha, svm_rec_joy_alpha, svm_f1_sadness_alpha, svm_f1_pleasure_alpha, svm_f1_anger_alpha, svm_f1_joy_alpha], [subject_id, 5, "STFT", "alpha", "-", "kNN", knn_acc_alpha, knn_prec_sadness_alpha, knn_prec_pleasure_alpha, knn_prec_anger_alpha, knn_prec_joy_alpha, knn_rec_sadness_alpha, knn_rec_pleasure_alpha, knn_rec_anger_alpha, knn_rec_joy_alpha, knn_f1_sadness_alpha, knn_f1_pleasure_alpha, knn_f1_anger_alpha, knn_f1_joy_alpha], [subject_id, 5, "STFT", "beta", "-", "Random Forest", rf_acc_beta, rf_prec_sadness_beta, rf_prec_pleasure_beta, rf_prec_anger_beta, rf_prec_joy_beta, rf_rec_sadness_beta, rf_rec_pleasure_beta, rf_rec_anger_beta, rf_rec_joy_beta, rf_f1_sadness_beta, rf_f1_pleasure_beta, rf_f1_anger_beta, rf_f1_joy_beta], [subject_id, 5, "STFT", "beta", "-", "SVM", svm_acc_beta, svm_prec_sadness_beta, svm_prec_pleasure_beta, svm_prec_anger_beta, svm_prec_joy_beta, svm_rec_sadness_beta, svm_rec_pleasure_beta, svm_rec_anger_beta, svm_rec_joy_beta, svm_f1_sadness_beta, svm_f1_pleasure_beta, svm_f1_anger_beta, svm_f1_joy_beta], [subject_id, 5, "STFT", "beta", "-", "kNN", knn_acc_beta, knn_prec_sadness_beta, knn_prec_pleasure_beta, knn_prec_anger_beta, knn_prec_joy_beta, knn_rec_sadness_beta, knn_rec_pleasure_beta, knn_rec_anger_beta, knn_rec_joy_beta, knn_f1_sadness_beta, knn_f1_pleasure_beta, knn_f1_anger_beta, knn_f1_joy_beta], [subject_id, 5, "STFT", "gamma", "-", "Random Forest", rf_acc_gamma, rf_prec_sadness_gamma, rf_prec_pleasure_gamma, rf_prec_anger_gamma, rf_prec_joy_gamma, rf_rec_sadness_gamma, rf_rec_pleasure_gamma, rf_rec_anger_gamma, rf_rec_joy_gamma, rf_f1_sadness_gamma, rf_f1_pleasure_gamma, rf_f1_anger_gamma, rf_f1_joy_gamma], [subject_id, 5, "STFT", "gamma", "-", "SVM", svm_acc_gamma, svm_prec_sadness_gamma, svm_prec_pleasure_gamma, svm_prec_anger_gamma, svm_prec_joy_gamma, svm_rec_sadness_gamma, svm_rec_pleasure_gamma, svm_rec_anger_gamma, svm_rec_joy_gamma, svm_f1_sadness_gamma, svm_f1_pleasure_gamma, svm_f1_anger_gamma, svm_f1_joy_gamma], [subject_id, 5, "STFT", "gamma", "-", "kNN", knn_acc_gamma, knn_prec_sadness_gamma, knn_prec_pleasure_gamma, knn_prec_anger_gamma, knn_prec_joy_gamma, knn_rec_sadness_gamma, knn_rec_pleasure_gamma, knn_rec_anger_gamma, knn_rec_joy_gamma, knn_f1_sadness_gamma, knn_f1_pleasure_gamma, knn_f1_anger_gamma, knn_f1_joy_gamma]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment5_1.csv", mode="a", header=False, index=False)